## Loading Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import gurobipy as gp
from gurobipy import GRB



## 1. Loading the data

In [ ]:
#read csv into dataframe
input = pd.read_csv("data_input.csv")

#read distance matrix into dataframe
distance = pd.read_csv("Distance_Matrix.csv", index_col=0, header=0)

In [ ]:
input

## 2. Optimization Section

In [ ]:
# Set up Gurobi environment
env = gp.Env(empty=True)
env.setParam('OutputFlag', 1)
env.start()

# Initialize the model
m = gp.Model(env=env)

### 2.1 Setting Data paremeters

In [ ]:
#create data frame for only 2018 and 2019 data
forecast = input[['Latitude','Longitude','2018','2019']] # F_it i refers to the index 0 to 2417 and t refers to the column 2018 and 2019
forecast = forecast.head(300)
distance
distance =  distance.iloc[0:300,0:300]
depot_cap = 20000
refinary_cap = 100000

### 2.2 Create Variables for Gurobi

In [ ]:
# Create binary variable for depot location
y = m.addVars(forecast.index, vtype=GRB.BINARY, name="y")

# Create binary variable for refinary location
z = m.addVars(forecast.index, vtype=GRB.BINARY, name="z")

# Create a matrix variable for flow from biamass production to depot for each year 2018 and 2019
B18 = m.addVars(forecast.index, forecast.index, vtype=GRB.CONTINUOUS, name="B18")
# for now 2019 is skipped

# Create a matrix variable for flow from depot to refinary for each year 2018 and 2019
P18 = m.addVars(forecast.index, forecast.index, vtype=GRB.CONTINUOUS, name="P18")
# for now 2019 is skipped

### 2.3 Constraints

In [ ]:
#1 All variables are positive 
m.addConstrs((B18[i,j] >= 0 for i in forecast.index for j in forecast.index), name="c1")
m.addConstrs((P18[i,j] >= 0 for i in forecast.index for j in forecast.index), name="c2")

#2 Biomass shipment from each index to any depot is less than the forecast for each year
m.addConstrs((B18.sum(i,'*') <= forecast.loc[i,'2018']  for i in forecast.index), name="c3") #forecast.loc[i,'2018']
# need to add for 2019 as B19

#3 Biomass shipment from each index to any depot must be less than depot capacity and only if its active
m.addConstrs((B18.sum('*',j) <= depot_cap * y[j] for j in forecast.index), name="c4")
# need to add for 2019 as B19

#4 Biomass shipment from each depot to refinary must be less than refinary capacity and only if its active
m.addConstrs((P18.sum('*',k) <= refinary_cap * z[k] for k in forecast.index), name="c5")
# need to add for 2019 as P19

#Totall depots cant be more than 25
m.addConstr((y.sum() <= 25), name="c6")

#Total refinary cant be more than 5
m.addConstr((z.sum() <= 5), name="c7")

#Total shipment from depot to reginary must be atleast 80% of total foreacast for 2018
m.addConstr((P18.sum() >= 0.8 * forecast['2018'].sum()), name="c8")
# need to add for 2019 as P19

# Flow constraint. The flow out of deport must be within 0.1% of the flow out of deport
m.addConstrs(( P18.sum(j,'*') == B18.sum('*',j) * 1 for j in forecast.index), name="c9")
# need to add for 2019 as P19

#m.addConstrs(( P18.sum(j,'*') <= B18.sum('*',j) * 1 for j in forecast.index), name="c9")
#m.addConstrs(( P18.sum(j,'*') >= B18.sum('*',j) * 0.999 for j in forecast.index), name="c10")


### 2.4 Objective Function

In [ ]:
# objective function is the sum of distance from index to deport and depott to refinary
m.setObjective(
    (gp.quicksum(B18[i,j] * distance.iloc[i,j] for i in forecast.index for j in forecast.index) + 
    gp.quicksum(P18[i,j] * distance.iloc[i,j] for i in forecast.index for j in forecast.index) +
    sum(depot_cap * y[j] for j in forecast.index) - B18.sum() + 
    sum(refinary_cap * z[k] for k in forecast.index) - P18.sum()    
     )  , GRB.MINIMIZE)

# Need to add for 2019 as P19 and B19

In [ ]:
# Update and write the model
m.update() # Update model parameters
m.write("Shell.lp") # Write model to file

### 2.5 Optimize

In [ ]:
    m.optimize()
    
    print("\nObjective value: ", m.getAttr("ObjVal"))

### 2.6 Makign the csv file

In [ ]:
#write into csv with specific column names
output = pd.DataFrame(columns=['year','data_type','source_index','destination_index','value'])
depot_plot = pd.DataFrame(columns=['index','value'])
refinary_plot = pd.DataFrame(columns=['index','value'])
#make csv from output
output.to_csv('output.csv', index=False)
# append rows into csv



In [ ]:
#print solution of vatriable y ~Depot
solution = m.getAttr('x',y)

for v in solution:
    if solution[v] > 0:
        #print(v, solution[v])
        #add as row to output csv
        output.loc[len(output)] = [20182019, 'depot_location', v,'', solution[v]]
        depot_plot.loc[len(depot_plot)] = [v, solution[v]]

output.to_csv('output.csv', index=False)

In [ ]:
#print solution of vatriable z ~Refinary
solution = m.getAttr('x',z)

for v in solution:
    if solution[v] > 0:
        #print(v, solution[v])
        output.loc[len(output)] = [20182019, 'refinery_location', v,'', solution[v]]
        refinary_plot.loc[len(refinary_plot)] = [v, solution[v]]

output.to_csv('output.csv', index=False)

In [ ]:
# add forecast of 2018 and 2019 into the csv file
for i in forecast.index:
    output.loc[len(output)] = [2018, 'biomass_forecast', i,'', forecast.loc[i,'2018']]
for i in forecast.index:
    output.loc[len(output)] = [2019, 'biomass_forecast', i,'', forecast.loc[i,'2019']]

output.to_csv('output.csv', index=False)

In [ ]:
#print solution of vatriable B ij ~Biomass moved from i to j
solution = m.getAttr('x',B18)

for v in solution:
    if solution[v] > 0:
        #print(v[0],v[1], solution[v])
        output.loc[len(output)] = [2018, 'biomass_demand_supply', v[0],v[1], solution[v]]

#repeat for 2019
#solution = m.getAttr('x',B18) Later add B19 to this

for v in solution:
    if solution[v] > 0:
        #print(v[0],v[1], solution[v])
        output.loc[len(output)] = [2019, 'biomass_demand_supply', v[0],v[1], solution[v]]

output.to_csv('output.csv', index=False)

In [ ]:
#print solution of vatriable P ij ~Biomass moved from i to j
solution = m.getAttr('x',P18)

for v in solution:
    if solution[v] > 0:
        #print(v[0],v[1], solution[v])
        output.loc[len(output)] = [2018, 'pellet_demand_supply', v[0],v[1], solution[v]]

#repeat for 2019
#solution = m.getAttr('x',P18) Later add B19 to this

for v in solution:
    if solution[v] > 0:
        #print(v[0],v[1], solution[v])
        output.loc[len(output)] = [2019, 'pellet_demand_supply', v[0],v[1], solution[v]]

output.to_csv('output.csv', index=False)

### 2.7 Visualize Map and Locations

In [ ]:
# add latitude and longitude from forecast to depot_plot dataframe
for i in depot_plot.index:
    depot_plot.loc[i,'Latitude'] = forecast.loc[int(depot_plot.loc[i,'index']),'Latitude']
    depot_plot.loc[i,'Longitude'] = forecast.loc[int(depot_plot.loc[i,'index']),'Longitude']

#depot_plot   

In [ ]:
# add latitude and longitude from forecast to refinary_plot dataframe
for i in refinary_plot.index:
    refinary_plot.loc[i,'Latitude'] = forecast.loc[int(depot_plot.loc[i,'index']),'Latitude']
    refinary_plot.loc[i,'Longitude'] = forecast.loc[int(depot_plot.loc[i,'index']),'Longitude']

#refinary_plot

In [ ]:
#plot latitude on x axis and longiture on y axis and plot each point of foreacst into a graph and add a color scale  
#forecast = input[['Latitude','Longitude','2018','2019']]

plt.figure(figsize=(8,6))

plt.scatter(forecast['Longitude'], forecast['Latitude'], c=forecast['2018'],cmap='cool', edgecolors='k', s=15) #binary or rainbow https://matplotlib.org/stable/tutorials/colors/colormaps.html
#add depot and refinary location into scatter plot as red and black points
plt.scatter(depot_plot['Longitude'], depot_plot['Latitude'], c='r', edgecolors='k', s=50)
plt.scatter(refinary_plot['Longitude'], refinary_plot['Latitude'], c='k', edgecolors='k', s=50)

# Add colorbar
cbar = plt.colorbar()
cbar.set_label('Color Scale')

# Customize the plot (labels, title, etc.)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Biomass forecase for 2018')
plt.xlim(68,75)
plt.ylim(20,25)
plt.show()
